In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install onnxruntime python-telegram-bot --upgrade

In [11]:
# 🔹 Устанавливаем библиотеки
!pip install onnxruntime python-telegram-bot --upgrade

# 🔹 Импортируем всё нужное
import os
import io
import numpy as np
import onnxruntime as ort
from PIL import Image
import torch
from torchvision import transforms
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext

# 🔹 Вставляем токен от BotFather
TELEGRAM_BOT_TOKEN = "7902861516:AAG8Gvf2IaQT6E5ZeluN0OzbgjiR81WAJq4"

# 🔹 Путь до модели ONNX
onnx_model_path = "/content/drive/MyDrive/best_model_finetuned.onnx"

# 🔹 Загружаем ONNX модель
ort_session = ort.InferenceSession(onnx_model_path)

# 🔹 Названия классов (казахские блюда)
class_names = ['ásip', 'baýyrsaq', 'beshbarmaq', 'irimshik', 'manty', 'naryn',
               'naýryz_kóje', 'orama', 'plain_soup', 'plov', 'qattama nan', 'qazy-qarta',
               'qurt', 'qymyz', 'qýyrdaq', 'samsa', 'sheep_head', 'shelpek', 'shorpa',
               'súzbe', 'taba-nan', 'talqan-jent']

# 🔹 Подготавливаем трансформации
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 🔹 Функция для предсказания
def predict_image(image: Image.Image) -> str:
    img_tensor = transform(image).unsqueeze(0).numpy()
    outputs = ort_session.run(None, {"input": img_tensor})
    pred_idx = np.argmax(outputs[0], axis=1)[0]
    return class_names[pred_idx]

# 🔹 Обработчик команды /start
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text("Sálem! Maǵan qazaq taǵamynyń sýretin jiberińiz, men ony anyqtaýǵa tyrysamyn. 😊")

# 🔹 Обработчик получения фото
async def handle_photo(update: Update, context: CallbackContext) -> None:
    photo = await update.message.photo[-1].get_file()
    photo_bytes = await photo.download_as_bytearray()

    image = Image.open(io.BytesIO(photo_bytes)).convert("RGB")

    predicted_class = predict_image(image)

    await update.message.reply_text(f"Meniń oıymsha, bul: *{predicted_class}*", parse_mode="Markdown")

# 🔹 Основная функция запуска бота
async def main():
    app = Application.builder().token(TELEGRAM_BOT_TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.PHOTO, handle_photo))

    print("✅ Bot iske qosyldy! Oǵan Telegram arqyly sýret jiberińiz.")

    await app.initialize()
    await app.start()
    await app.updater.start_polling()

# 🔹 Запуск
import asyncio
await main()


ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 658, in get_updates
    updates = await super().get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", line 4593, in get_updates
    await self._post(
  File "/usr/local/lib/python3.11/dist-packages/tele

✅ Bot iske qosyldy! Oǵan Telegram arqyly sýret jiberińiz.
